<a href="https://colab.research.google.com/github/RSAMUEL12/WebScraper/blob/master/multiWebScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from google.colab import files
import pandas as pd
import datetime

In [0]:
#Used to extract data from the specified website holding a table of race results.
#Function is specifically used to handle data from THAT PARTICULAR WEBSITE but can be altered to target different table data
def extractData(soup, element):
    rows = soup.find_all("tr", class_="ms-table_row")
    row_data = []
    for row in rows:
      data = row.find_all("td", class_=element)
      str_data = str(data)
      cleantext = BeautifulSoup(str_data, "lxml").get_text()
      cleantext = cleantext.strip("[]\n")
      cleantext = cleantext.strip()
      row_data.append(cleantext)
    
    row_data.pop(0)
    row_data = row_data[:20]
    return row_data


In [0]:
#Used to alter the "times" list to add DNF results
def addDNF(list):
  new_list = []
  for each in list:
    if each == "":
      each = "DNF"
    new_list.append(each)
    
  return new_list

In [0]:
def toDateTime(list):
  new_list = []
  for each in list:
    each = each.replace(".", "")
    time_dt = datetime.datetime.strptime(each, "%M:%S'%f")
    new_list.append(time_dt)
  return new_list

In [0]:
def removeNullPoints(list):
  new_list = []
  for each in list:
    if each == "":
      each = "0"
    new_list.append(each)
  return new_list

In [0]:
### MAIN PROGRAM ###
#URLs set up for the web scraping
main = "https://www.motorsport.com"
url = "https://www.motorsport.com/f1/results/"
html = urlopen(url)

#Creating the web scraper with BeautifulSoup4
soup = BeautifulSoup(html)

In [75]:
#The Main Feature of this Python notebook is that multiple links are searched with BeautifulSoup
#These links are added to a list and iterated through by having them concatenated with the root link of the website
gp_links = []
for link in soup.find_all('a', class_='ms-results-subnav_item'):
  print(link.get('href'))
  gp_links.append(link.get('href'))

/f1/results/2019/australian-gp-417660/
/f1/results/2019/bahrain-gp/
/f1/results/2019/chinese-gp/
/f1/results/2019/azerbaijan-gp/
/f1/results/2019/spanish-gp-417684/
/f1/results/2019/monaco-gp-417689/
/f1/results/2019/canadian-gp-417694/
/f1/results/2019/french-gp-417699/
/f1/results/2019/austrian-gp-417705/
/f1/results/2019/british-gp-417710/
/f1/results/2019/german-gp/
/f1/results/2019/hungarian-gp-417720/
/f1/results/2019/belgian-gp-417725/
/f1/results/2019/italian-gp-417730/
/f1/results/2019/singapore-gp-417735/
/f1/results/2019/russian-gp-417740/
/f1/results/2019/japanese-gp-417745/


In [0]:
#Need to loop through all existing race results
count = 1
name = "race"

for each in gp_links:
  url = main + each
  html = urlopen(url)
  soup = BeautifulSoup(html)

  table = createTable(soup)
  dataframe = createDataFrame(table)
  #download the dataframe as a file
  #downloadDT(dataframe, name)
  count+=1

  

In [0]:
#Each of the attributes and its values are collected using the extractData() function and uses the 'element' variable to specify the HTML ID it is contained within
def createTable(soup):
  element = "ms-table_cell ms-table_field--"
  names = extractData(soup, element + "result_driver_id")
  number = extractData(soup, element + "number")
  position = addDNF(extractData(soup, element + "pos"))
  car_make = extractData(soup, element + "car_make")
  times = extractData(soup, element + "time")
  points = removeNullPoints(extractData(soup, element + "points"))

  #Dictionary to hold the data and create a Pandas Dataframe
  race_results = {"Position": position, "Driver Name": names, "Driver No.": number, "Team": car_make, "Times": times, "Points": points}

  return race_results;

In [0]:
def createDataFrame(table):
  results_pd = pd.DataFrame(table)
  return results_pd

In [0]:
def downloadDT(dataframe, name):
  dataframe.to_csv('filename.csv') 
  files.download('filename.csv')  

In [10]:
#String to DateTime
#Tenth and hundreth of the second are stored as microseconds in DateTime Objects
#by stripping the decimal point and having '27325 in the microsecond format -> 000000, 000001,...,999999 microseconds
time = times[0]
time = time.replace(".", "")
print(time)
time_dt = datetime.datetime.strptime(time, "%M:%S'%f")
print(time_dt.microsecond)

1:25'27325
273250
